<a href="https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/Query_CTR-FCEN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Búsquedas WOS+SCI+SCP+PTJ+CTR para FCEN

Búsquedas en bases bibligráficas  
* Web of Science (WOS), 
* Scielo (SCI)
* Scopus  (SCP)
* Puntaje (UDEA)
* Center (CTR)
de los artículos científicos de la UdeA

La base de datos se creó con:

[WOS_SCI_SCP_PTJ_CTR.ipynb](./WOS_SCI_SCP_PTJ_CTR.ipynb)

In [178]:
import os
VERSION='NEW'
if os.getcwd()=='/content':
    !pip install openpyxl xlrd wosplus fuzzywuzzy[speedup] > /dev/null

## functions

In [1]:
import pandas as pd
import wosplus as wp

##  Configure public links of  files in Google Drive
* If it is a Google Spreadsheet the corresponding file is downloaded as CSV
* If it is in excel/json or text file the file is downloaded  directly

To define your  own labeled IDs for public google drive files edit the next cell:

In [2]:
%%writefile drive.cfg
[FILES]
WOS_SCI_SCP_PTJ_CTR.json.gz=19E1C1kRk4I0V3uXojqko8-NEicWaPp1j
UDEA_authors_with_WOS_info.json=1o1eVT4JD0FMMICq_oxrTJOzWh47veBMw

Overwriting drive.cfg


##  Load data bases

In [3]:
drive_files=wp.wosplus('drive.cfg')

In [4]:
UDEAjsonfile='WOS_SCI_SCP_PTJ_CTR.json.gz'
tmp=drive_files.load_biblio(UDEAjsonfile,compression='gzip')
UDEA=drive_files.biblio['WOS'].copy().reset_index(drop=True)

In [32]:
#UDEA=pd.read_json('WOS_SCI_SCP_PTJ_CTR.json.gz',compression='gzip')

In [7]:
json_column='UDEA_authors'
UDEA_YES=UDEA[UDEA['UDEA_authors']!=''].reset_index(drop=True)
UDEA_NOT=UDEA[UDEA['UDEA_authors']==''].reset_index(drop=True)
UDEA_YES.shape[0],UDEA_NOT.shape[0]

(12324, 3376)

# Búsquedas

In [8]:
import pandas as pd
import numpy as np
import fuzzywuzzy.process as fwp
from fuzzywuzzy import fuzz
def flatten_if_nested(l):
    flatten=False
    for i in l:
        if type(i)==list:
            #return i
            flatten=True
    if flatten:
        l=[item for sublist in l for item in sublist]
        l=pd.np.array(l)
    return l
def extract_key(df,key,json_column='UDEA_authors'):
    '''
    Extract all the unique key values of the list of dictionaries in 
    a json column when the key value is a string or another list
    '''
    ll=df[json_column].apply(lambda l: np.unique([ d.get(key) for d in l 
                                if d.get(key) ]) if type(l)==list else l)
    if ll.str[0].apply(lambda l: l if type(l)==list else None).dropna().shape[0]:
        ll=ll.apply(flatten_if_nested)
    ll=ll.apply(pd.Series).stack().values
    return pd.DataFrame( {key:list(ll)} ).groupby(key)[key].count().sort_values(ascending=False)
def extract_key_unique(*args,**kwargs):
    keys=extract_key(*args,**kwargs).keys()
    return [ k for k in keys if k]




## Cree indices para búsquedas

In [9]:
def get_groups(l,g):
    '''Normalize columna GRUPO'''
    for d in l:
        gt=d.get('GRUPO')
        if gt and type( gt )==str:
            gs=gt.replace(
                ', Grupo','; Grupo'
            ).split('; ')
            for gg in gs:
                if gg not in g:
                    g.append(gg)
    return g

facultades={'key':'FACULTAD',
            'values' : extract_key_unique(UDEA,'FACULTAD',json_column='UDEA_authors') }
departamentos={'key':'DEPARTAMENTO',
            'values' :extract_key_unique(UDEA,'DEPARTAMENTO',json_column='UDEA_authors')}
nombre_completo={'key'    : 'NOMBRE COMPLETO',
            'values' : extract_key_unique(UDEA,'NOMBRE COMPLETO',json_column='UDEA_authors')}
full_name={'key'    : 'full_name',
            'values' : extract_key_unique(UDEA,'full_name',json_column='UDEA_authors')}
udea_affiliations={'key'    : 'WOS_affiliation',
            'values' : extract_key_unique(UDEA.fillna(''),'WOS_affiliation',json_column='UDEA_authors')}
#wos_affiliations={'key'    : 'affiliation',
#            'values' : extract_key_unique(UDEA.fillna(''),'WOS_affiliation',json_column='authors_WOS')}
udea_author={'key'    : 'WOS_author',
            'values' : extract_key_unique(UDEA.fillna(''),'WOS_author',json_column='UDEA_authors')}
wos_author={'key'    : 'WOS_author',
            'values' : extract_key_unique(UDEA,'WOS_author',json_column='authors_WOS')}

In [12]:
def query_json_column(q,df=UDEA,json_column='UDEA_authors',
                        choices=nombre_completo,scorer=fuzz.partial_token_sort_ratio,score_cutoff=0):
    #Found best exact match from index
    fchoices=fwp.extractOne(q,choices['values'],scorer=scorer,score_cutoff=score_cutoff)
    # Exact search in indexed subcolumn converted to strins (e.g list → string if necessary)
    if fchoices:
        fchoices=fchoices[0]
        dfF=df[df[json_column].apply(lambda l: True in [ str(d.get(choices['key'])).find(fchoices)>-1 
                                        for d in l if d.get(choices['key'])] if type(l)==list else False)]
        return dfF.reset_index(drop=True)
    else:
        return pd.DataFrame()

## Centro

In [13]:
cen=query_json_column('Facultad de Ciencias Exactas y Naturales',df=UDEA,json_column='UDEA_authors',
                        choices=facultades,scorer=fuzz.partial_token_sort_ratio,score_cutoff=79)

In [14]:
cen.shape

(2415, 181)

## Departamento

In [15]:
extract_key(cen,'DEPARTAMENTO')[:4]

DEPARTAMENTO
Instituto de Física            916
Instituto de Biología          680
Instituto de Química           666
Departamento de Matemáticas    190
Name: DEPARTAMENTO, dtype: int64

In [16]:
aunly=drive_files.read_drive_json('UDEA_authors_with_WOS_info.json').reset_index(drop=True)

In [17]:
aunly['DEPARTAMENTO']=aunly['UDEA_authors'].apply( lambda d: d.get('DEPARTAMENTO')   ).dropna()

In [18]:
dpto=['Instituto de Física',
        'Instituto de Biología',
        'Instituto de Química',
        'Departamento de Matemáticas' ]

In [19]:
print('DEPARTAMENTO')
for k in dpto: 
    print('{}:  {}'.format(k,
                    aunly[aunly['DEPARTAMENTO']==k].shape[0]))

DEPARTAMENTO
Instituto de Física:  35
Instituto de Biología:  34
Instituto de Química:  37
Departamento de Matemáticas:  24


### CITACIONES

In [20]:
df=pd.DataFrame()
dfinst={}
for k in dpto:
    inst={}
    dfinst[k]=query_json_column(k,df=UDEA,json_column='UDEA_authors',
                        choices=departamentos,scorer=fuzz.partial_token_sort_ratio,score_cutoff=100)
    inst['DEPARTAMENTO']=k
    inst['WOS Cited by']=dfinst[k].Z9.sum()
    inst['SCP Cited by']=dfinst[k]['SCP_Cited by'].sum()
    df=df.append(inst,ignore_index=True)

ct=[c for c in df.columns if c.find('Cited')>-1]
for c in ct:
    df[c]=df[c].astype(int)
df.loc[0,'GS Cited by*']='22500'
print('* https://scholar.google.com/citations?user=mxSOjTYAAAAJ')
df    

* https://scholar.google.com/citations?user=mxSOjTYAAAAJ


,DEPARTAMENTO,SCP Cited by,WOS Cited by,GS Cited by*
0,Instituto de Física,8349,8227,22500
1,Instituto de Biología,6898,11603,NaN
2,Instituto de Química,7089,7078,NaN
3,Departamento de Matemáticas,492,384,NaN


In [21]:
pd.set_option('display.max_rows', 500)

In [22]:
extract_key(dfinst['Instituto de Física'],'full_name')

full_name
DUQUE ECHEVERRI CARLOS ALBERTO         261
MORALES ARAMBURO ALVARO LUIS            86
BARRERA RAMIREZ JOHN FREDY              62
RESTREPO CARDENAS JOHANS                60
BARRERO MENESES CESAR AUGUSTO           59
MORA RAMOS MIGUEL EDUARDO               49
REYES GOMEZ ERNESTO AMADOR              49
SANZ VICARIO JOSE LUIS                  46
ARNACHE OLMOS OSCAR LUIS                40
RESTREPO QUINTERO DIEGO ALEJANDRO       37
PONCE GUTIERREZ WILLIAM ANTONIO         34
HENAO HENAO RODRIGO DE JESUS            32
MAZO ZULUAGA JOHANN                     31
OSORIO GUILLEN JORGE MARIO              29
OSORIO VELEZ JAIME ALBERTO              26
RODRIGUEZ REY BORIS ANGHELO             25
ROMANO ANTONIO ENEA                     23
ZULUAGA CALLEJAS JORGE IVAN             23
RAIGOZA BOHORQUEZ NICOLAS FERNANDO      22
ZAPATA NOREÑA OSCAR ALBERTO             22
GARCIA TELLEZ KAREN EDILMA              20
LOPEZ RIOS SONIA YANETH                 19
RUEDA MUNOZ EDGAR ALBERTO               19
P

In [99]:
i=0
print(dpto[i])
aunly[aunly['DEPARTAMENTO']==dpto[i]].sort_values('tmp_author').reset_index(drop=True).tmp_author

Instituto de Física


0              ARNACHE OLMOS OSCAR LUIS
1            BARRERA RAMIREZ JOHN FREDY
2         BARRERO MENESES CESAR AUGUSTO
3         CUARTAS RESTREPO PABLO ANDRES
4        DUQUE ECHEVERRI CARLOS ALBERTO
5          FERRIN VAZQUEZ IGNACIO RAMON
6         GIRALDO CADAVID MARCO ANTONIO
7          HENAO HENAO RODRIGO DE JESUS
8       JARAMILLO ARANGO DANIEL ESTEBAN
9     JARAMILLO GALLEGO JOHNY ALEXANDER
10      LONDOÑO BADILLO FERNANDO ANDRES
11          MAHECHA GOMEZ JORGE EDUARDO
12                  MAZO ZULUAGA JOHANN
13               MIRA AGUDELO ALEJANDRO
14         MORALES ARAMBURO ALVARO LUIS
15            MUÑOZ CUARTAS JUAN CARLOS
16           OSORIO GUILLEN JORGE MARIO
17           OSORIO VELEZ JAIME ALBERTO
18    PACHON CONTRERAS LEONARDO AUGUSTO
19      PONCE GUTIERREZ WILLIAM ANTONIO
20             RESTREPO CARDENAS JOHANS
21    RESTREPO QUINTERO DIEGO ALEJANDRO
22           REYES GOMEZ ERNESTO AMADOR
23               RICAURTE AVELLA GERMAN
24          RODRIGUEZ REY BORIS ANGHELO


In [23]:
pd.set_option('display.max_colwidth',2000)

In [24]:
wp=UDEA[UDEA.AU.str.contains('Ponce, W')][['TI','SO','PY','AU','authors_WOS','UDEA_authors']].reset_index(drop=True)

In [28]:
wp.apply(lambda r: fwp.extractOne(  [ a for a in r['AU'].split('\n')[0:-1] 
                       if a.find('Ponce, W')>-1][0], list( aus),
                        scorer=fuzz.partial_token_sort_ratio) ,axis=1 )[:1]                                 

0    (Ponce, William A., 88)
dtype: object

### Others

In [31]:
for k in dpto:
    print('='*40)
    print(k)
    print('='*40)
    print( extract_key(dfinst[k],'full_name') )

Instituto de Física
full_name
DUQUE ECHEVERRI CARLOS ALBERTO         261
MORALES ARAMBURO ALVARO LUIS            86
BARRERA RAMIREZ JOHN FREDY              62
RESTREPO CARDENAS JOHANS                60
BARRERO MENESES CESAR AUGUSTO           59
MORA RAMOS MIGUEL EDUARDO               49
REYES GOMEZ ERNESTO AMADOR              49
SANZ VICARIO JOSE LUIS                  46
ARNACHE OLMOS OSCAR LUIS                40
RESTREPO QUINTERO DIEGO ALEJANDRO       37
PONCE GUTIERREZ WILLIAM ANTONIO         34
HENAO HENAO RODRIGO DE JESUS            32
MAZO ZULUAGA JOHANN                     31
OSORIO GUILLEN JORGE MARIO              29
OSORIO VELEZ JAIME ALBERTO              26
RODRIGUEZ REY BORIS ANGHELO             25
ROMANO ANTONIO ENEA                     23
ZULUAGA CALLEJAS JORGE IVAN             23
RAIGOZA BOHORQUEZ NICOLAS FERNANDO      22
ZAPATA NOREÑA OSCAR ALBERTO             22
GARCIA TELLEZ KAREN EDILMA              20
LOPEZ RIOS SONIA YANETH                 19
RUEDA MUNOZ EDGAR ALBERT

full_name
MONDRAGON PEREZ FANOR                  84
LOPEZ OSORIO BETTY LUCY                75
RESTREPO COSSIO ALBEIRO ALONSO         70
ECHEVERRI LOPEZ LUIS FERNANDO          54
TORRES PALMA RICARDO ANTONIO           54
LOPEZ LOPEZ DIANA PATRICIA             38
MESA CADAVID MONICA                    36
ECHAVARRIA ISAZA ADRIANA PATRICIA      32
QUIÑONES FLETCHER WINSTON              26
HADAD ARRIAGADA CACIER ZILAHY          25
PELAEZ JARAMILLO CARLOS ALBERTO        25
VAZQUEZ CEBALLOS MARIO VICTOR          23
CARDONA GALEANO WILSON ISIDRO          23
SANTAMARIA PALACIO ALEXANDER           22
MANRIQUE MORENO MARCELA MARIA          21
GARCIA TELLEZ KAREN EDILMA             21
OTALVARO TAMAYO LEON FELIPE            21
ESPINAL LOPEZ JUAN FERNANDO            21
ROBLEDO RESTREPO SARA MARIA            21
BARRERO MENESES CESAR AUGUSTO          20
PUERTAS MEJIA MIGUEL ANGEL             18
GALLEGO MARIN JAIME ANDRES             17
CASANOVA YEPES HERLEY FERNANDO         17
RAMIREZ VELEZ ALEJANDRO 

## TODO

BUG: 31

In [16]:
aw=UDEA[UDEA['authors_WOS'].apply(len)==0][['AU','authors_WOS','C1','UDEA_authors']].reset_index(drop=True)
aw[0:35]

,AU,authors_WOS,C1,UDEA_authors
0,"Armas, O\nBallesteros, R\nMartos, FJ\nAgudelo,...",[],"Univ Castilla La Mancha, Escuela Tecn Super In...",[{'NOMBRE COMPLETO': 'John Ramiro Agudelo Sant...
1,"Salazar-Maya, AM\nGuarin-Berrio, GE\nArroyave-...",[],"[Maria Salazar-Maya, Angela; Elena Guarin-Berr...","[{'PRIMER APELLIDO': 'Salazar', 'SEGUNDO APELL..."
2,"Beltran-Salazar, OA\n",[],"Univ Antioquia Medellin, Fac Enfermeria, Bogot...","[{'PRIMER APELLIDO': 'Beltran', 'SEGUNDO APELL..."
3,"Franco, NEM\n",[],"Univ Antioquia, Dept Trabjo Social, Fac Cienci...",[{'NOMBRE COMPLETO': 'Nora Eugenia Muñoz Franc...
4,"Lopez, JLO\n",[],"Univ Antioquia, Medellin, Colombia.\n","[{'PRIMER APELLIDO': 'Orduña', 'SEGUNDO APELLI..."
5,"Blair, S\n",[],"Univ Antioquia, Grp Malaria, Sede Invest Univ,...","[{'PRIMER APELLIDO': 'Blair', 'SEGUNDO APELLID..."
6,"Penaranda-Correa, F\n",[],"Univ Antioquia, Fac Nacl Salud Publ, Medellin,...","[{'PRIMER APELLIDO': 'Peñaranda', 'SEGUNDO APE..."
7,"Arias Lopez, BE\n",[],"Univ Antioquia, Fac Enfermeria, Medellin, Colo...","[{'PRIMER APELLIDO': 'Arias', 'SEGUNDO APELLID..."
8,"Orozco-Arroyave, JR\nBelalcazar-Bolanos, EA\nA...",[],"[Rafael Orozco-Arroyave, Juan; Alexander Belal...",[{'NOMBRE COMPLETO': 'Juan Rafael Orozco Arroy...
9,"Salazar-Maya, AM\n",[],"Univ Antioquia, Medellin, Colombia.\n","[{'PRIMER APELLIDO': 'Salazar', 'SEGUNDO APELL..."


In [263]:
aw.shape

(2055, 4)

Strategy: 
* From `full name` in `UDEA_authors` column creates `WOS_author` list.
* Check if some author in `C1` column is in that list
* Creates keys:values for keys: `WOS_author` and `[WOS_]affiliations` in both `authors_WOS` and `UDEA_authors`
* Assign affiliation from `C1` if not ambiguity or left a generic one: `affil`
* Assign a tag to `authors_WOS` to specify the origin, e.g, {'origin':'AU'}

In [38]:
from wos_sci_scp_ptj_ctr import *

In [39]:
import numpy as np

In [47]:
affil='Univ Antioquia'

In [118]:
s='Restrepo Quintero, ÓÑ'


'Restrepo-Quintero, Ó. Ñ.'

In [306]:
def AU_to_author_WOS(wos_au,au_info,c1_old,affil='Univ Antioquia',extra_affil=', Medellín, Colombia'):
    """
    From the information of 'AU' and 'C1' of a WOS database, generates de 'authors_WOS' list
    of dictionaries
    """
    AUWOS=False
    AFFIL=False
    wn=wos_names_list(au_info)
    wn=wn+[ re.sub( 'Mc(\w)',lambda s: 'Mc'+s.group(1).upper(),n)  
            for n in wn if n.find('.')>-1]    
    wn=wn+[ re.sub( 'Mc(\w)',lambda s: 'Mc'+s.group(1).upper(), 
                   n.replace('. ','.').replace('.',''))  
            for n in wn if n.find('.')>-1]
    wn=wn+[ re.sub( 'Mc(\w)',lambda s: 'Mc'+s.group(1).upper(), 
                   n.replace('. ','.').replace('.','').replace('-',' '))  
            for n in wn if n.find('.')>-1]
    wos_author_list=wos_au.strip().split('\n')
    mtch=np.intersect1d(wos_author_list,wn)
    #Try again without UNICODE characters:
    if len(mtch)==0:
        wos_au=unidecode.unidecode(wos_au)
        wos_author_list=wos_au.strip().split('\n')
        mtch=np.intersect1d(wos_author_list,wn)
    #Finally matched!:
    if len(mtch)==1:
        AUWOS=True
        full_affs=c1_old.strip().split('\n') 
        affs=[ c1 for c1 in full_affs 
                 if c1.find(affil)>-1]
        #Cases:
        # 1. Single match
        if len(affs)==1:
            AFFIL=True
            aff=affs
        elif len(affs)>1:
            # 2 Ordered mathc
            try: 
                #exact match or first one: idx_au=0
                idx_au=wos_author_list.index(mtch[0])
                if full_affs[idx_au].find(affil)>-1:
                    AFFIL=True
                    aff=[full_affs[idx_au]]
            except:
                try:
                    #last one:
                    idx_au=wos_author_list.index(mtch[0])
                    if idx_au==len(wos_author_list)-1:
                        if full_affs[-1].find(affil)>-1:
                            AFFIL=True
                            aff=[full_affs[-1]]
                except:
                    pass
        # 3. One of the very similar matches
        
        
    auwos={}    
    if AUWOS:
        #renormalize to WOS C1 standars
        wos_author=re.sub('^(\w+)\s(\w+,)'               ,r'\1-\2'  ,
                   re.sub('(\s[A-ZÁÉÍÓÚÑ])([A-ZÁÉÍÓÚÑ])$',r'\1. \2.',
                   re.sub('(\s[A-ZÁÉÍÓÚÑ])$'             ,r'\1.',mtch[0],
                                                             re.UNICODE),
                                                             re.UNICODE),
                                                             re.UNICODE)
        auwos['WOS_author']=wos_author
        auwos['origin']='from AU+UDEA_authors'
        if AFFIL:
            auwos['affiliation']=aff
        else:
            auwos['affiliation']=[affil+extra_affil]
    return auwos

def authors_Wos_from_bad_AU_and_bad_C1(row,authors_WOS_col='authors_WOS',
                                           AU_col='AU',C1_col='C1',
                                           UDEA_authors_col='UDEA_authors'):
    '''
    Use poor normalized "AU" and "C1" columns from WOS to 
    generate proper normalized "authors_WOS column"
    '''
    if row[authors_WOS_col]:
        return row[authors_WOS_col]
    
    awl=row[UDEA_authors_col]
    author_wos_list=[]
    if awl:

        for j in range(len(awl)):
            auudea=dict(awl[j])
            auwos=AU_to_author_WOS( row[AU_col],auudea,row[C1_col])
            if auwos and auwos not in author_wos_list:
                author_wos_list=author_wos_list+[auwos]
    return author_wos_list

In [307]:
#Prepate UDEA file
UDEA['authors_WOS']=UDEA['authors_WOS'].apply(lambda l: l if type(l)==list else [])

before:

In [308]:
UDEA[UDEA['authors_WOS'].apply(lambda l:len(l))==0].shape

(2055, 181)

In [309]:
import pandas as pd
import swifter

In [310]:
kk=UDEA.swifter.apply(lambda row: authors_Wos_from_bad_AU_and_bad_C1(row), axis=1)

After

In [311]:
UDEA['authors_WOS']=kk

In [312]:
UDEA[UDEA['authors_WOS'].apply(lambda l:len(l))==0].shape

(1099, 181)